# Brain Mapping Pipeline - Overview
## Overview
This is a pipeline to analyze brain mapping images. The pipeline was written with zebrafish and _Astyanax_ whole brain images stained with tERK and pERK antibodies and imaged using confocal microscopy. The pipeline follows MAPMapping from the Schier/Engert lab (Randlett et al., Nature Methods, 2015, PMID: 26778924). The pipeline gives a Python implementation ands functionality for analysis.

This notebook is meant to highlight the general flow.

### Imports
Aside from numpy and matplotlib, you'll need the brain_mapping and brain_mapping_tools. The main containers for experiments and the experiment suite is held in brain_mapping.py and the tools for stack manipulation, preprocessing and permutation is held in brain_mapping_tools.

In [1]:
import os, re
import numpy as np
import matplotlib.pyplot as plt
import brainmaps as bmps

### Get images

Optional step. This just lists the files in your directory stripping the _01 and _02 placeholder.
The pipeline expects a set of files in .nrrd format from ImageJ/FIJI. Channel one (usually tERK) is appended with _01 and channel two (usually pERK) is appended with _02. Thus, an experiment called `BrainExperiment` will have two files: `BrainExperiment_01.nrrd` and `BrainExperiment_02.nrrd`. As of now, the pipeline only handles up to two channels (one is fine), but analysis can be done on either channel, one channel normalized to the other, or on log differences. The file loader is fairly flexible, but it assumes that the filename before the _01|_02 is the name of the trial, and it expects it to be in .nrrd format. It will recognize these types of inputs:

`brain_one_01.nrrd` and `brain_one_2.nrrd`
or
`brain_one_01_warp_1g3462-1e6.nrrd` and `brain_one_2_warp_1g3462-1e6.nrrd`

In either case, it is assumed that your trail name is `brain_one` (which the config file expects to be explicitly stated) and that there is either only a `_01` file or both a `_01` and `_02` file.

In [2]:
exps = set()
for item in os.listdir("brainscans/"):
    if matches := re.search(r"([\w]+)_\d{2}.nrrd$", item):
        exps.add(matches.group(1))
exps

{'SPF2_SF_LightDark_no5', 'SPF2_SF_LightDark_no6', 'SPF2_SF_LightDark_no7'}

### Load pipeline
This step sets up your experiment. You'll need the file path - in this case one string that removes _01.nrrd and _02.nrrd as its assumed the preceeding text indicates the shared file. You'll define the path, what the condition is. zfirst is a FLAG that described indexing. NumPy can index on the first dimension fastest, though ImageJ usually saves in (x,y,z) format. If zfirst is set to False, the pipeline switches the z-dimension so that the result is (z,x,y). If zfirst is set to True it leaves it alone, but assume its already in (z,x,y). Direction can be reversed at any time using `Experiment.flip_axis()` or `AnalysisSuite.flip_axes()`.

In [3]:
configs = [
    bmps.Config(filepath="brainscans/SPF2_SF_LightDark_no5", condition="Surface_LD", zfirst=False),
    bmps.Config(filepath="brainscans/SPF2_SF_LightDark_no5", condition="Pach_LD", zfirst=False),
]

suite = bmps.AnalysisSuite.from_configs(configs)

### Pre-processing
#### Check
Before running the analysis, you'll need to ensure that the number of conditions, z-direction, and dimensions are all OK. There is a FLAG that will not let you proceed until this is done. It can be called implicitly later on, but its best to ensure all parameters check out here first

In [4]:
suite.check_anlysis_conditions()

All parameters check out. Proceed to setup analysis run


#### Select groups
Make a Tuple of strings that define the conditions. You can see what conditions are availavle by either checking your load function above, or running `suite.get_conditons()`

In [5]:
groups = ("Surface_LD", "Pach_LD")

#### Prepare your groups.
This is the main pre-processing step. You will define a few things here:
1. Define the 'suite' variable. This is whatever you called `bmap.AnalysisSuite.from_configs(configs)`
2. What groups to use. This parallels the above.
3. What channel would you like the analysis to be run on? You can run on channel 1 or channel 2, normalized channel 1 by channel 2, normalized channel 2 by channel 1 (most similar to Randlett), or the log ratios of either channel.
4. Do you want to normalize the brightness? This can help offset any differences you set in laser power among images.

In [13]:
# channel_mode options: "ch1", "ch2", "ndi_ch1", "ndi_ch2", "log_ratio_ch1", "log_ratio_ch2"
prepared = bmps.prepare_for_permutation(
    suite,
    groups=groups,
    channel_mode="ndi_ch2",      
    do_brightness_affine=True,     
    downsample = True,
    ds_factors = (2,2,2),
    ds_method = "local_mean"
)

(106, 512, 973)
(106, 512, 973)


### Run your permutation. 
Specify the prepared file you made in preprocessing, groups, number of permutations, tails, alpha and a random state.

In [15]:
res = bmps.run_permutation(prepared,groups = groups, n_perm = 2000, tail = "two-sided", alpha = 0.05)

100%|████████████████████████████████████| 2000/2000 [16:40:54<00:00, 30.03s/it]


In [12]:
print(res)

PermutationResult
  Groups: n1=1, n2=1
  Effect map shape: (106, 256, 487)
  Params: tail=two-sided, n_perm=500, alpha=0.05, seed=None
  P-values: min=1, median=1, max=1
  Q-values: min=1, median=1, max=1
  Significant voxels: 0/13215232 (0.00%) at q <= 0.05



## Advanced use
Oftentimes times you will have large datasets and making config files for these can be difficult and time-consuming.
brainmaps comes equipped with a helper-function that allows you to build config files from larger datasets. 

Assume you have an experiment that has young and aged brain, with both drug and placebo. Each condition has several nrrd files. You may envision a directory structure like this:

```
root_folder/
├── young_control/
│   ├── brain_yc_01.nrrd
│   ├── brain_yc_02.nrrd
│   ├── brain_yc_03_01.nrrd
│   ├── brain_yc_03_02.nrrd
│
├── young_experimental/
│   ├── brain_ye_01.nrrd
│   ├── brain_ye_02.nrrd
│   ├── brain_ye_03_01.nrrd
│   ├── brain_ye_03_02.nrrd
│
├── aged_control/
│   ├── brain_ac_01.nrrd
│   ├── brain_ac_02.nrrd
│   ├── brain_ac_03_01.nrrd
│   ├── brain_ac_03_02.nrrd
│
├── aged_experimental/
│   ├── brain_ae_01.nrrd
│   ├── brain_ae_02.nrrd
│   ├── brain_ae_03_01.nrrd
│   ├── brain_ae_03_02.nrrd
```

You can make a config file using the following method:
`build_configs_from_directory("/data/root_folder", zfirst=False)`. The output will be a config file that look like this:

```
configs = [
    Config(
        filepath="/data/root_folder/young_control/brain_yc_01",
        condition="young_control",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/young_control/brain_yc_02",
        condition="young_control",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/young_control/brain_yc_03",
        condition="young_control",
        zfirst=False
    ),

    Config(
        filepath="/data/root_folder/young_experimental/brain_ye_01",
        condition="young_experimental",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/young_experimental/brain_ye_02",
        condition="young_experimental",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/young_experimental/brain_ye_03",
        condition="young_experimental",
        zfirst=False
    ),

    Config(
        filepath="/data/root_folder/aged_control/brain_ac_01",
        condition="aged_control",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/aged_control/brain_ac_02",
        condition="aged_control",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/aged_control/brain_ac_03",
        condition="aged_control",
        zfirst=False
    ),

    Config(
        filepath="/data/root_folder/aged_experimental/brain_ae_01",
        condition="aged_experimental",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/aged_experimental/brain_ae_02",
        condition="aged_experimental",
        zfirst=False
    ),
    Config(
        filepath="/data/root_folder/aged_experimental/brain_ae_03",
        condition="aged_experimental",
        zfirst=False
    ),
]
```

You can also write a json with the flag: `write_json="/path/configs.json"`

`build_configs_from_directory("/data/root_folder", zfirst=False, write_json="/path/configs.json)`

In [ ]:
configs = build_configs_from_directory("/data/root_folder", zfirst=False, write_json="~/Desktop/configs.json)
suite = bmps.AnalysisSuite.from_configs(configs)                                   